In [29]:

import pandas as pd
import plotly.express as px
import numpy as np
from tqdm import tqdm
from pathlib import Path
folder = Path('/Users/fcf/Downloads/parallel_mlps/experiments/exp0082_fixed_test_1m1l_internalsbss_holdout/')
folder.absolute()

PosixPath('/Users/fcf/Downloads/parallel_mlps/experiments/exp0082_fixed_test_1m1l_internalsbss_holdout')

In [31]:
df_all_path = folder / "df_all.csv"
if not df_all_path.exists():
	filepaths = list(folder.glob("**/pmlps_1.csv"))
	df = pd.DataFrame()
	list_dfs = []
	for filepath in tqdm(filepaths):
		splits = str(filepath).split('/')

		tmp_df = pd.read_csv(filepath)
		tmp_df["dataset"] = splits[-4]
		tmp_df["run"] = splits[-2]
		list_dfs.append(tmp_df)
	df = pd.concat(list_dfs, ignore_index=True)
	df.to_csv(folder / "df_all.csv")

df = pd.read_csv(df_all_path)
print(df.shape)
df.head()


100%|██████████| 197/197 [00:24<00:00,  7.95it/s]


(1103200, 24)


,Unnamed: 0,model_id,selected,fold,repetition,num_neurons,weight_regularization,monitored_metric,loss,epoch,...,validation_overall_acc,validation_matthews_corrcoef,holdout_overall_acc,holdout_matthews_corrcoef,test_overall_acc,test_matthews_corrcoef,best_arch_id,dominant_solution,dataset,run
0,0,1271,False,4,4,72,0.000137,0.985380,0.117150,94,...,0.985380,0.967453,0.965562,0.923586,0.964912,0.921423,False,True,car(3),18
1,1,2088,False,4,4,89,0.000123,0.982456,0.112033,97,...,0.982456,0.960874,0.966862,0.926390,0.959064,0.908362,False,True,car(3),18
2,2,1289,False,4,4,90,0.000119,0.979532,0.118214,94,...,0.979532,0.954163,0.964912,0.921905,0.959064,0.908362,False,False,car(3),18
3,3,1267,False,4,4,68,0.000141,0.979532,0.122945,97,...,0.979532,0.954153,0.961663,0.914449,0.964912,0.921423,False,True,car(3),18
4,4,2059,False,4,4,60,0.000143,0.976608,0.126700,98,...,0.976608,0.947519,0.961663,0.914687,0.964912,0.921423,False,True,car(3),18


In [28]:
# Calculating Differences
df["diffs"] = (
	abs(df["train_overall_acc"] - df["validation_overall_acc"])
	+ abs(df["train_overall_acc"] - df["holdout_overall_acc"])
	+ abs(df["validation_overall_acc"] - df["holdout_overall_acc"])
) / 3
# Filtering models

num_neurons = [3, 5, 10, 15, 20, 50]
activatinos = ["Identity()", "ReLU()"]

df_filtered = df[df["num_neurons"].isin(num_neurons) & df["activation_name"].isin(activatinos)]

# calculating differences
df_filtered
print(df_filtered.shape)
df_filtered.groupby(["dataset", "num_neurons", "activation_name"])["train_overall_acc", "validation_overall_acc", "holdout_overall_acc", "test_overall_acc"].agg(["mean", "std"])

(18912, 25)


/var/folders/fm/wtnz2xgd26l7pj0nsb5zc35c0000gn/T/ipykernel_98174/748049483.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_filtered.groupby(["dataset", "num_neurons", "activation_name"])["train_overall_acc", "validation_overall_acc", "holdout_overall_acc", "test_overall_acc"].agg(["mean", "std"])


train_overall_acc            \
                                                 mean       std   
dataset num_neurons activation_name                               
0       3           Identity()               0.721708  0.197916   
                    ReLU()                   0.699958  0.199686   
        5           Identity()               0.754475  0.154841   
                    ReLU()                   0.736901  0.165270   
        10          Identity()               0.763754  0.147967   
...                                               ...       ...   
18      15          ReLU()                   0.780867  0.141545   
        20          Identity()               0.782944  0.125244   
                    ReLU()                   0.799833  0.123949   
        50          Identity()               0.802106  0.119063   
                    ReLU()                   0.829549  0.117975   

                                    validation_overall_acc            \
                                                      mean       std   
dataset num_neurons activation_name                                    
0       3           Identity()                    0.756706  0.166194   
                    ReLU()                        0.722644  0.191500   
        5           Identity()                    0.781429  0.131942   
                    ReLU()                        0.746441  0.164998   
        10          Identity()                    0.793755  0.113985   
...                                                    ...       ...   
18      15          ReLU()                        0.800846  0.112793   
        20          Identity()                    0.812595  0.096825   
                    ReLU()                        0.810942  0.106511   
        50          Identity()                    0.826193  0.082170   
                    ReLU()                        0.829643  0.102846   

                                    holdout_overall_acc            \
                                                   mean       std   
dataset num_neurons activation_name                                 
0       3           Identity()                 0.750448  0.169105   
                    ReLU()                     0.717692  0.193391   
        5           Identity()                 0.777618  0.129993   
                    ReLU()                     0.745229  0.162146   
        10          Identity()                 0.789783  0.112178   
...                                                 ...       ...   
18      15          ReLU()                     0.800700  0.107411   
        20          Identity()                 0.808585  0.089867   
                    ReLU()                     0.812809  0.102078   
        50          Identity()                 0.825501  0.078184   
                    ReLU()                     0.835040  0.105344   

                                    test_overall_acc            
                                                mean       std  
dataset num_neurons activation_name                             
0       3           Identity()              0.741890  0.183030  
                    ReLU()                  0.707285  0.201642  
        5           Identity()              0.763914  0.152510  
                    ReLU()                  0.735323  0.175629  
        10          Identity()              0.772807  0.139721  
...                                              ...       ...  
18      15          ReLU()                  0.781170  0.126535  
        20          Identity()              0.788150  0.121908  
                    ReLU()                  0.790263  0.126079  
        50          Identity()              0.806237  0.097689  
                    ReLU()                  0.804228  0.117633  

[216 rows x 8 columns]